In [1]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as ureq
from selenium import webdriver
import time
import re

In [2]:
url = "https://calendar.engineering.queensu.ca/preview_program.php?catoid=9&poid=503&returnto=232"

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--incognito')
#chrome_options.add_argument('--headless')

driver = webdriver.Chrome("C:\\Users\\jerry\\Downloads\\chromedriver", options=chrome_options)

In [24]:
driver.get(url)

# 1. Collect all course link texts for webdriver to click on

In [5]:
page_soup = soup(driver.page_source, 'lxml')

In [6]:
containers = page_soup.find("div", {"class": "acalog-core"}).findAll("a")
len(containers)

16

In [13]:
link_texts = [container.text.strip() for container in containers if len(container.text)>0]
link_texts

['APSC 100 Engineering Practice l FW |  K9',
 'APSC 199 English Proficiency for Engineers FW, S |  K0.2',
 'APSC 111 Physics I F (6WK Sess2) | 3.3',
 'APSC 131 Chemistry and Materials F (6WK Sess1) | 3.3',
 'MNTC 313 Introduction to Programming F/S-O/L | 3',
 'APSC 151 Earth Systems Engineering F (6WK Sess2) | 3.3',
 'APSC 171 Calculus I F (6WK Sess1) | 3.3',
 'APSC 112 Physics II W | 3.3',
 'APSC 114 NOT OFFERED 2020-2021 Electricity and Magnetism W | 3.3',
 'APSC 132 Chemistry and its Applications W | 3.3',
 'APSC 162 Engineering Graphics W | 2.5',
 'APSC 172 Calculus II W | 3.3',
 'APSC 174 Introduction to Linear Algebra W | 3.3',
 'APSC 182 Applied Engineering Mechanics W, F | 1.7']

In [22]:
link_texts = [link_text.replace("  ", " ") for link_text in link_texts]
link_texts

['APSC 100 Engineering Practice l FW | K9',
 'APSC 199 English Proficiency for Engineers FW, S | K0.2',
 'APSC 111 Physics I F (6WK Sess2) | 3.3',
 'APSC 131 Chemistry and Materials F (6WK Sess1) | 3.3',
 'MNTC 313 Introduction to Programming F/S-O/L | 3',
 'APSC 151 Earth Systems Engineering F (6WK Sess2) | 3.3',
 'APSC 171 Calculus I F (6WK Sess1) | 3.3',
 'APSC 112 Physics II W | 3.3',
 'APSC 114 NOT OFFERED 2020-2021 Electricity and Magnetism W | 3.3',
 'APSC 132 Chemistry and its Applications W | 3.3',
 'APSC 162 Engineering Graphics W | 2.5',
 'APSC 172 Calculus II W | 3.3',
 'APSC 174 Introduction to Linear Algebra W | 3.3',
 'APSC 182 Applied Engineering Mechanics W, F | 1.7']

# 2. Click open all course info boxes

In [25]:
counter = 0
for link_text in link_texts:
    link = driver.find_element_by_link_text(link_text)
    time.sleep(2)
    link.click()
    time.sleep(3)
    counter += 1
    
print(counter)

14


# 3. Obtain updated page html and scrape all course info

In [26]:
page_soup = soup(driver.page_source, 'lxml')

In [27]:
containers = page_soup.find("div", {"class": "acalog-core"}).findAll("li", {"class": "acalog-course acalog-course-open"})
len(containers)

14

In [41]:
course_titles = [re.split(" [FW]", container.find("div", {"class": None}).h3.text)[0] for container in containers]
course_titles

['APSC 100 Engineering Practice l',
 'APSC 199 English Proficiency for Engineers',
 'APSC 111 Physics I',
 'APSC 131 Chemistry and Materials',
 'MNTC 313 Introduction to Programming',
 'APSC 151 Earth Systems Engineering',
 'APSC 171 Calculus I',
 'APSC 112 Physics II',
 'APSC 114 NOT OFFERED 2020-2021 Electricity and Magnetism',
 'APSC 132 Chemistry and its Applications',
 'APSC 162 Engineering Graphics',
 'APSC 172 Calculus II',
 'APSC 174 Introduction to Linear Algebra',
 'APSC 182 Applied Engineering Mechanics']

In [42]:
course_codes = [" ".join(course_title.split()[:2]) for course_title in course_titles]
course_codes

['APSC 100',
 'APSC 199',
 'APSC 111',
 'APSC 131',
 'MNTC 313',
 'APSC 151',
 'APSC 171',
 'APSC 112',
 'APSC 114',
 'APSC 132',
 'APSC 162',
 'APSC 172',
 'APSC 174',
 'APSC 182']

In [43]:
course_names = [" ".join(course_title.split()[2:]) for course_title in course_titles]
course_names

['Engineering Practice l',
 'English Proficiency for Engineers',
 'Physics I',
 'Chemistry and Materials',
 'Introduction to Programming',
 'Earth Systems Engineering',
 'Calculus I',
 'Physics II',
 'NOT OFFERED 2020-2021 Electricity and Magnetism',
 'Chemistry and its Applications',
 'Engineering Graphics',
 'Calculus II',
 'Introduction to Linear Algebra',
 'Applied Engineering Mechanics']

In [52]:
re.split("  Academic Units", re.split("Tutorial: [YesNo.0-9]{1,3}", containers[0].text)[1])[0]

'This course introduces fundamental professional engineering skills and provides an opportunity to apply engineering science and mathematics content in situations emulating professional practice. It consists of three modules: Module 1. Problem analysis and modeling; Module 2. Experimentation and measurement; Module 3: Engineering design. The course provides an introduction to personal learning styles, team dynamics, oral and written presentation skills, laboratory data collection, analysis and presentation, project management, information management, problem analysis and modeling, numeric computation, economics, design methodologies, and workplace safety.'

In [57]:
course_descs = [re.split("Tutorial: [YesNo.0-9]{1,4}", container.text)[1].split(" Academic Units:")[0].replace("\xa0", "").strip() for container in containers]
course_descs

['This course introduces fundamental professional engineering skills and provides an opportunity to apply engineering science and mathematics content in situations emulating professional practice. It consists of three modules: Module 1. Problem analysis and modeling; Module 2. Experimentation and measurement; Module 3: Engineering design. The course provides an introduction to personal learning styles, team dynamics, oral and written presentation skills, laboratory data collection, analysis and presentation, project management, information management, problem analysis and modeling, numeric computation, economics, design methodologies, and workplace safety.',
 'This course develops skills that are necessary to organize and present technical information in a professional context. At the end of the course students will demonstrate English proficiency in listening comprehension and written expression.',
 'This course is an introduction to Newtonian mechanics in the context of engineering a

# 4. Write to CSV

In [58]:
import pandas as pd

df = pd.DataFrame({
    "Course Number": course_codes,
    "Course Name": course_names,
    "Course Description": course_descs    
})

df

,Course Number,Course Name,Course Description
0,APSC 100,Engineering Practice l,This course introduces fundamental professiona...
1,APSC 199,English Proficiency for Engineers,This course develops skills that are necessary...
2,APSC 111,Physics I,This course is an introduction to Newtonian me...
3,APSC 131,Chemistry and Materials,This course provides an introduction to the ch...
4,MNTC 313,Introduction to Programming,Students will be introduced to the fundamental...
5,APSC 151,Earth Systems Engineering,This course provides an introduction to the co...
6,APSC 171,Calculus I,Graphs and derivatives of vector-valued functi...
7,APSC 112,Physics II,This course continues from APSC 111to introduc...
8,APSC 114,NOT OFFERED 2020-2021 Electricity and Magnetism,This course continues from APSC 111 to introdu...
9,APSC 132,Chemistry and its Applications,This course combines fundamentals of chemistry...


In [ ]:
df.to_csv('QueensU_Engineering_Common_First_Year_Courses.csv', index = False)